In [6]:
import numpy as np
import pandas as pd
import datetime as dt

# Encoding and Data Split
# from sklearn.preprocessing import FeatureHasher
import category_encoders as ce
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer


# Plotting Moudles
import matplotlib.pyplot as plt


#Modeling
from sklearn import metrics
import scipy.stats as stats
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

data = pd.read_csv('Data/stitchedGuide.csv')
partnerData = pd.read_csv('Data/partners.csv')

# Setting random seed for reproducibility
np.random.seed(0)

data.head()

,Org Name,EIN,Current FY Start,Current FY End,Ruling Year,Form Year,Form Type,Address Line 1,Address Line 2,City,...,Legal Fees,Pension Plan Contributions Employee Benefits,Printing and Publications,Professional Fundraising Expenses,Principal Officer,Org Leader,Primary Contact Name,Primary Contact Title,Primary Contact Email,Number of Employees
0,Marjorie F Campbell Educational Trust,10-6006537,2018-01-01,2018-12-31,2007.0,2018.0,990PF,PO Box 1810,NaN,Sanford,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Friends of Westons Council On Aging Inc,11-0016148,2018-07-01,2019-06-30,1990.0,2018.0,990EZ,PO Box 672,NaN,Weston,...,NaN,NaN,"$5,423",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Connection Fund, Inc.",11-0017051,2017-07-01,2018-06-30,1990.0,2017.0,990,955 S Main St,NaN,Middletown,...,"$217,833",NaN,NaN,"$10,180",LISA DEMATTEIS-LEPORE,Peter Nucci,Heide Erb,Communications Coordinator,herb@theconnectioninc.org,63.0
3,Jhd Jr All of Our Children Memorial Fund,11-0025081,NaN,NaN,1990.0,NaN,NaN,771 Broadway,NaN,Paterson,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mvnhc Inc,11-0026102,NaN,NaN,1990.0,NaN,NaN,107 W 4th St,NaN,Mount Vernon,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
np.shape(data)

(3000, 66)

Adding Independent Variable Column

### Removing Rows and Columns
* Rows removed based on duplicates of EIN and organization name
    * Rows also moved based on missing ruling year or extremely old


In [8]:
# Trying to set 'Ruling Year' into an integer and removing all rows that have years < 1900
df = data.copy()
df['Ruling Year'].fillna(0).astype(int)
df = df[df['Ruling Year'] >= 1900]

partnerData['Ruling Year'].fillna(0).astype(int)
partnerData = partnerData[partnerData['Ruling Year'] >= 1900]

# Dropping duplicates based on 'EIN' and 'Org Name'
df.drop_duplicates(subset = ['EIN', 'Org Name'], inplace = True)


In [9]:
# Dropping Unnecessary Columns
df = df[['State', 'Subject Area 1', 'Total Assets', 'Total Revenue','Number of Employees']]

# Dropping Same Columns for partnerData
partnerData = partnerData[['State', 'Subject Area 1', 'Total Assets', 'Total Revenue','Number of Employees']]

In [10]:
df.tail(10000)

,State,Subject Area 1,Total Assets,Total Revenue,Number of Employees
0,ME,Education,"$142,119","$5,640",NaN
1,MA,Senior services,"$156,917","$49,225",NaN
2,CT,Community and economic development,"$11,383,533","$10,003,333",63.0
3,NJ,Youth services,NaN,NaN,NaN
4,NY,Homeless services,NaN,NaN,NaN
...,...,...,...,...,...
2993,NY,Judaism,NaN,NaN,NaN
2994,NY,Cultural awareness,NaN,NaN,NaN
2995,NY,Business and industry,NaN,NaN,NaN
2996,NY,Racquet sports,NaN,NaN,NaN


### Handling Missing Variables

In [11]:
# Checking for Missing Data
missing_val = df.isnull().sum()
print(missing_val[0:])

State                     4
Subject Area 1            3
Total Assets           1735
Total Revenue          1743
Number of Employees    2130
dtype: int64


In [12]:
# Imputing Subject Area
df ['Subject Area 1'] = df['Subject Area 1'].fillna('U')
# df ['Subject Area 2'] = df['Subject Area 2'].fillna('U')
# df ['Subject Area 3'] = df['Subject Area 3'].fillna('U')

# Imputing State
df ['State'] = df['State'].fillna('U')

missing_val = df.isnull().sum()
print(missing_val[5:])

Series([], dtype: int64)


In [13]:
# Imputing w/ 0 for Total Assets, Revenue, and # of Employees
df['Total Assets'] = df['Total Assets'].str.replace(',', '').str.replace('$', '').fillna(0).astype(int)
df['Total Revenue'] = df['Total Revenue'].str.replace(',', '').str.replace('$', '').str.extract('(\d+)').fillna(0).astype(int)
df['Number of Employees'] = df['Number of Employees'].fillna(0).astype(int)


# Imputing w/ 0 for Total Assets, Revenue, and # of Employees
partnerData['Total Assets'] = partnerData['Total Assets'].str.replace(',', '').str.replace('$', '').fillna(0).astype(int)
partnerData['Total Revenue'] = partnerData['Total Revenue'].str.replace(',', '').str.replace('$', '').str.extract('(\d+)').fillna(0).astype(int)
partnerData['Number of Employees'] = partnerData['Number of Employees'].fillna(0).astype(int)



In [14]:
df['partner'] = 0
partnerData['partner'] = 1

merge = pd.concat([df, partnerData])
df.dtypes

State                  object
Subject Area 1         object
Total Assets            int64
Total Revenue           int64
Number of Employees     int64
partner                 int64
dtype: object

In [15]:
merge.head()


,State,Subject Area 1,Total Assets,Total Revenue,Number of Employees,partner
0,ME,Education,142119,5640,0,0
1,MA,Senior services,156917,49225,0,0
2,CT,Community and economic development,11383533,10003333,63,0
3,NJ,Youth services,0,0,0,0
4,NY,Homeless services,0,0,0,0


### Data Preparation
* Categorical Encoding
* Train Test Splitting the Data



In [16]:
# Dividing the Dataset into Features(X) and Target (y)
X = merge.drop(columns = ['partner'])
y = merge['partner']

# Setting up the pipeline
import joblib
location = 'cache'
memory = joblib.Memory(location=location, verbose=10)



encoding_pipeline = Pipeline([
    ('encode_category', ce.HashingEncoder(cols=['Subject Area 1'], return_df=True)),
    ('encode_other', ce.OneHotEncoder(cols=['State'], return_df=True)),
], memory=memory)

In [17]:
# Train-Test-Splitting the Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [18]:
X_train_encoded = encoding_pipeline.fit_transform(X_train, y_train)
X_test_encoded = encoding_pipeline.transform(X_test)

________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(HashingEncoder(cols=['Subject Area 1'], max_process=2),      State    Subject Area 1  Total Assets  Total Revenue  Number of Employees
532     NY      Christianity        705618          16807                    0
1749    MO    Human services             0              0                    0
144     NY         Education        307174         408152                   13
689     NY         Education             0              0                    0
1807    NY  Arts and culture             0              0                    0
...    ...               ...           ...            ...                  ...
770     NY      Human rights         43185         224807                    3
842     NY    Human services        360951        1016710                    0
1668    TX    Youth services             0              0                    

In [19]:
print(X_train_encoded.shape)
print(X_test_encoded.shape)

(2193, 63)
(549, 63)


In [20]:
X_test_encoded.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,State_1,State_2,...,State_46,State_47,State_48,State_49,State_50,State_51,State_52,Total Assets,Total Revenue,Number of Employees
0,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,74829,0,0


## Decision Tree Regression/Random Forest

In [21]:
dtr = DecisionTreeRegressor()
dtr.fit(X_train_encoded,y_train)

DecisionTreeRegressor()

In [22]:
# Calculating Error
mae = metrics.mean_absolute_error(y_test,dtr.predict(X_test_encoded))
mse = metrics.mean_squared_error(y_test,dtr.predict(X_test_encoded))
rmse = np.sqrt(metrics.mean_squared_error(y_test,dtr.predict(X_test_encoded)))
r2 = metrics.r2_score(y_test, dtr.predict(X_test_encoded))

print("mae: ", mae)
print("mse: ", mse)
print("rmse: ", rmse)
print("r2: ", r2)

mae:  0.01092896174863388
mse:  0.01092896174863388
rmse:  0.10454167469786334
r2:  -0.5110091743119265


In [23]:
rfr = RandomForestRegressor()
rfr.fit(X_train_encoded,y_train)

RandomForestRegressor()

In [24]:
mae = metrics.mean_absolute_error(y_test,rfr.predict(X_test_encoded))
mse = metrics.mean_squared_error(y_test,rfr.predict(X_test_encoded))
rmse = np.sqrt(metrics.mean_squared_error(y_test,rfr.predict(X_test_encoded)))
r2 = metrics.r2_score(y_test, rfr.predict(X_test_encoded))

In [25]:
print("mae: ", mae)
print("mse: ", mse)
print("rmse: ", rmse)
print("r2: ", r2)

mae:  0.012641165755919854
mse:  0.005761020036429873
rmse:  0.075901383626584
r2:  0.20349669724770636


In [26]:
# Compute feature importances
pd.DataFrame({'feature':X_test_encoded.columns, 
              'importance':rfr.feature_importances_}).sort_values(by='importance',ascending=False)

,feature,importance
61,Total Revenue,0.369744
60,Total Assets,0.222795
62,Number of Employees,0.125446
21,State_14,0.082481
1,col_1,0.038621
...,...,...
9,State_2,0.000000
33,State_26,0.000000
34,State_27,0.000000
35,State_28,0.000000
